In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
  temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
  separator="\n",
  chunk_size=600,
  chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
  embeddings, cache_dir
)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriver = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
  [
    (
      "system",
      "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}"
    ),
    ("human", "{question}")
  ]
)

chain = (
  {
    "context": retriver,
    "question": RunnablePassthrough(),
  }
  | prompt
  | llm
)

chain.invoke("Victory Mansions을 묘사해줘")



AIMessage(content='Victory Mansions은 윈스턴 스미스가 살고 있는 아파트입니다. 그 아파트는 계단으로 오르내리는 7층 건물이며, 리프트는 거의 작동하지 않습니다. 아파트 복도는 삶은 양배추와 오래된 러그 매트의 냄새가 났고, 벽에는 실내 전시용으로는 너무 큰 컬러 포스터가 붙어 있습니다. 아파트 내부에는 전기 스위치로 조절되는 텔레스크린이 있으며, 윈스턴은 창가로 이동하여 바깥 풍경을 바라봅니다. 아파트는 경제 절약을 위해 주간에는 전기가 차단되는 등 여러 가지 제약이 있습니다.')